In [2]:
import torch
from torch import nn
from torch.utils.data import random_split 
import torchvision
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
from torchmetrics import Accuracy
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch import Trainer

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [2]:
batch_size = 64
NUM_WORKERS = int(os.cpu_count() / 2)

transform = transforms.Compose(
    [transforms.Resize((128, 128)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
DATA_DIR = "../inaturalist_12K"
train_dataset_complete = torchvision.datasets.ImageFolder(root=os.path.join(DATA_DIR, "train"), transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root=os.path.join(DATA_DIR, "val"), transform=transform)

# Splitting train into train and valid
valid_size = int(0.2 * len(train_dataset_complete))
train_size = len(train_dataset_complete) - valid_size 

train_dataset, val_dataset = random_split(train_dataset_complete, [train_size, valid_size])

# Dataloader
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=NUM_WORKERS)

val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=NUM_WORKERS)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=NUM_WORKERS)

classes = train_dataset_complete.classes
NUM_CLASSES = len(classes)

AttributeError: module 'torch' has no attribute '_six'

In [4]:
def imgplot(data):
  images, labels = next(iter(data))
  plt.figure(figsize=(10,8))
  samples = []
  for i in range(len(images)):
    img = images[i]/2 + 0.5
    if labels[i] in samples:
      continue
    else:
      samples.append(labels[i])
      plt.subplot(2,5,len(samples))
      plt.imshow(img.permute(1,2,0))
      plt.axis('off')
      plt.title(classes[labels[i]])
    if len(samples) == NUM_CLASSES:
      break
  plt.tight_layout()
  plt.show()


imgplot(train_dataloader)

NameError: name 'train_dataloader' is not defined

In [132]:
model = CNN(10)
model.convblock5(model.convblock4(model.convblock3(model.convblock2(model.convblock1(next(iter(train_dataloader))[0][0]))))).shape

torch.Size([48, 1, 1])

In [152]:
class CNN(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super().__init__()
        self.convblock1 = torch.nn.Sequential(
            nn.Conv2d(3, 3, 4),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2))
        self.convblock2 = torch.nn.Sequential(
            nn.Conv2d(3, 6, 4),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2))
        self.convblock3 = torch.nn.Sequential(
            nn.Conv2d(6, 12, 4),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2))
        self.convblock4 = torch.nn.Sequential(
            nn.Conv2d(12, 24, 4),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2))
          
        self.convblock5 = torch.nn.Sequential(
            nn.Conv2d(24, 48, 4),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=1))
        self.fc = nn.Linear(48, 512)
        self.out = nn.Linear(512, num_classes)
        
    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.out(x)
        return x


In [158]:
class CNN_light(pl.LightningModule):
    def __init__(self, lr =0.05):
        super().__init__()
        self.save_hyperparameters()        
        self.model = CNN()
        self.train_accuracy = Accuracy(task='multiclass', num_classes=NUM_CLASSES)
        self.val_accuracy = Accuracy(task='multiclass', num_classes=NUM_CLASSES)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        acc = self.train_accuracy(logits, y)
        print("train loss: ", loss.item())
        print("train accuracy: ", acc.item())
        self.log("train loss", loss)
        self.log("train accuracy", acc)

        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        acc = self.train_accuracy(logits, y)
        self.log("val loss", loss)
        self.log("val accuracy", acc)

        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.hparams.lr, momentum=0.9)
        return optimizer

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7e3e1905e550>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7e3e0c89f490, raw_cell="class CNN_light(pl.LightningModule):
    def __ini.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/user/Documents/Courses/DA6401_A2/lightning_test.ipynb#X22sZmlsZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7e3e1905e550>> (for post_run_cell), with arguments args (<ExecutionResult object at 7e3e0d02e6d0, execution_count=158 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7e3e0c89f490, raw_cell="class CNN_light(pl.LightningModule):
    def __ini.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/user/Documents/Courses/DA6401_A2/lightning_test.ipynb#X22sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe